In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

In [2]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [3]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [4]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [12]:
params = {
    "n_estimators": [100, 200, 300, 500],
    "max_samples": [0.5, 0.7, 0.9, 1.0],
    "max_features": [0.5, 0.7, 0.9, 1.0],
    "bootstrap": [True, False],
    "bootstrap_features": [True, False]
}

bag = BaggingClassifier(random_state = 2025)

In [13]:
np.random.seed(2025)
bag_grid = GridSearchCV(bag, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
bag_grid.fit(X_temp, y_temp)
print('Best Parameters:', bag_grid.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.7, 'max_samples': 0.7, 'n_estimators': 100}


In [14]:
np.random.seed(2025)
bag_model = BaggingClassifier(n_estimators = 100, max_samples = 0.7, max_features = 0.7, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model.fit(X_temp, y_temp)
y_pred = bag_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [15]:
np.random.seed(2025)
bag_model.fit(X_full, y)
y_pred_bag = bag_model.predict(X_test_full)

In [16]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_bag})

with open('BAG_Full_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [17]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [18]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [19]:
np.random.seed(2025)
bag_grid_rfe = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
bag_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', bag_grid_rfe.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 500}


In [20]:
np.random.seed(2025)
bag_model_rfe = BaggingClassifier(n_estimators = 500, max_samples = 1.0, max_features = 0.5, bootstrap = False, bootstrap_features = False, random_state = 2025)
bag_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = bag_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [21]:
np.random.seed(2025)
bag_model_rfe.fit(X_RFE, y)
y_pred_rfe_bag = bag_model_rfe.predict(X_RFE_test)

In [22]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_bag})

with open('BAG_RFE_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [23]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [24]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [25]:
np.random.seed(2025)
bag_grid_lasso = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
bag_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', bag_grid_lasso.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.9, 'max_samples': 0.9, 'n_estimators': 200}


In [26]:
np.random.seed(2025)
bag_model_lasso = BaggingClassifier(n_estimators = 200, max_samples = 0.9, max_features = 0.9, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = bag_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [27]:
np.random.seed(2025)
bag_model_lasso.fit(X_LASSO, y)
y_pred_lasso_bag = bag_model_lasso.predict(X_LASSO_test)

In [28]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_bag})

with open('BAG_LASSO_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [29]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [30]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [31]:
np.random.seed(2025)
bag_grid = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
bag_grid.fit(X_temp, y_temp)
print('Best Parameters:', bag_grid.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 300}


In [32]:
np.random.seed(2025)
bag_model = BaggingClassifier(n_estimators = 300, max_samples = 1.0, max_features = 0.5, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model.fit(X_temp, y_temp)
y_pred_bag_no = bag_model.predict(X_test)

print(classification_report(y_test, y_pred_bag_no))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [33]:
np.random.seed(2025)
bag_model.fit(X_full_no, y)
y_pred_bag_no = bag_model.predict(X_test_full_no)

In [34]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_bag_no})

with open('BAG_Full_No_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [35]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [36]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [37]:
np.random.seed(2025)
bag_grid = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
bag_grid.fit(X_temp, y_temp)
print('Best Parameters:', bag_grid.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 0.9, 'n_estimators': 100}


In [38]:
np.random.seed(2025)
bag_model = BaggingClassifier(n_estimators = 100, max_samples = 0.9, max_features = 1.0, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model.fit(X_temp, y_temp)
y_pred_manual = bag_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [39]:
np.random.seed(2025)
bag_model.fit(X_no_manual, y)
y_pred_bag_no_manual = bag_model.predict(X_test_no_manual)

In [40]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_bag_no_manual})

with open('BAG_No_Manual_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [41]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [42]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [43]:
np.random.seed(2025)
bag_grid = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
bag_grid.fit(X_temp, y_temp)
print('Best Parameters:', bag_grid.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 0.9, 'n_estimators': 300}


In [44]:
np.random.seed(2025)
bag_model = BaggingClassifier(n_estimators = 300, max_samples = 0.9, max_features = 1.0, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model.fit(X_temp, y_temp)
y_pred_manual = bag_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [45]:
np.random.seed(2025)
bag_model.fit(X_manual, y)
y_pred_bag_manual = bag_model.predict(X_manual_test)

In [46]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_bag_manual})

with open('BAG_Manual_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SMOTE FOR BEST BAG:

In [47]:
from imblearn.over_sampling import SMOTE

In [48]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

smote = SMOTE(random_state = 2025)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

In [49]:
np.random.seed(2025)
bag_grid_lasso = GridSearchCV(BaggingClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
bag_grid_lasso.fit(X_temp, y_temp)
print('Best Parameters:', bag_grid_lasso.best_params_)

Best Parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.9, 'max_samples': 0.9, 'n_estimators': 200}


In [50]:
np.random.seed(2025)
bag_model_smote = BaggingClassifier(n_estimators = 200, max_samples = 0.9, max_features = 0.9, bootstrap = False, bootstrap_features = True, random_state = 2025)
bag_model_smote.fit(X_resampled, y_resampled)
y_pred_smote = bag_model_smote.predict(X_test)

print(classification_report(y_test, y_pred_smote))
y_pred_smote_bag = bag_model_smote.predict(X_LASSO_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [51]:
from google.colab import files

pred_bag = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_smote_bag})

with open('BAG_SMOTE_Cov.csv', 'w') as file:
    pred_bag.to_csv(file, index = False, header = True)

files.download('BAG_SMOTE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GHOST:


In [52]:
!pip install ghostml

In [53]:
import ghostml
from sklearn import metrics

def calc_metrics(labels_test, test_probs, threshold = 0.5):
  scores = [1 if x>=threshold else 0 for x in test_probs]
  auc = metrics.roc_auc_score(labels_test, test_probs)
  kappa = metrics.cohen_kappa_score(labels_test, scores)
  confusion = metrics.confusion_matrix(labels_test, scores, labels = list(set(labels_test)))

  print('Threshold: %.2f, kappa: %.3f, AUC test set: %.3f'%(threshold, kappa, auc))
  print(confusion)
  print(metrics.classification_report(labels_test, scores))

  return

thresholds = np.round(np.arange(0.01, 0.75,0.01),2)

In [55]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

bag_ghost = BaggingClassifier(n_estimators = 200, max_samples = 0.9, max_features = 0.9, bootstrap = True, bootstrap_features = True, oob_score = True, random_state = 2025)
bag_ghost.fit(X_temp, y_temp)
bag_test_probs = bag_ghost.predict_proba(X_test)[:,1]
calc_metrics(y_test, bag_test_probs, threshold = 0.5)

Threshold: 0.50, kappa: 1.000, AUC test set: 1.000
[[36  0]
 [ 0  9]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [56]:
np.random.seed(2025)
bag_temp_probs = bag_ghost.predict_proba(X_temp)[:,1]
bag_threshold = ghostml.optimize_threshold_from_predictions(y_temp, bag_temp_probs, thresholds, ThOpt_metrics = 'Kappa')

np.random.seed(2025)
calc_metrics(y_test, bag_test_probs, threshold = bag_threshold)
bag_GHOST_test = [1 if x >= bag_threshold else 0 for x in bag_test_probs]

Threshold: 0.25, kappa: 1.000, AUC test set: 1.000
[[36  0]
 [ 0  9]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [57]:
np.random.seed(2025)
bag_ghost.fit(X_LASSO, y)
bag_probs = bag_ghost.predict_proba(X_LASSO_test)[:,1]
bag_pred_ghost = [1 if x >= bag_threshold else 0 for x in bag_probs]

In [58]:
from google.colab import files

pred_rf = pd.DataFrame({'ID': range(1, 92), 'Placement': bag_pred_ghost})

with open('BAG_GHOST_Cov.csv', 'w') as file:
    pred_rf.to_csv(file, index = False, header = True)

files.download('BAG_GHOST_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>